In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

# Pandas - Introduction

In this notebook we will familiarize us with Pandas, the todays de-facto library for data handling. Pandas is a big library designed on top of numpy for structured data, with statistical tools, convenient data importing functions and tools to filter and extract data.

A whole bunch of tutorials for Pandas are available online:
https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html<br>
or<br>
https://dataanalysispython.readthedocs.io/en/latest/index.html<br>
The latter beeing my personal favorite<br>

We will focus on a few functions of pandas:
1. The convenient way to organise and slice data
2. The import function, that has options for most types of data and will work nearly always
3. The very simple but powerful plot function
4. Grouping, sorting data and statistical tool

Pandas has three Basic structures:
* Series, 1 dimensional data with an index
* 2D data with index and columns
* DateTime Frame 2D dataframe with specialized index

If you slice a 2d Dataframe into a single column you have a pandas Series. All have a somewhat similar structure.

A DataFrame has the structure:
<div>
<img src="Data/DataFrameStructure.png" width="400">
</div>

One of the most important differences to e.g. arrays is that you can select/slice the data by name. Lets create a notebook and take a look


In [ ]:
pd.__version__

In [ ]:
import pandas as pd
import numpy as np

#create dictionary with data
data = {
    'Compound': ['Water', 'Ethanol', 'Benzene', 'Methanol', 'Acetone', 'Toluene', 'Tetrachloroethylene'],
    'MolecularWeight': [18.015, 46.07, 78.1134, 32.042, 58.08, 92.14, 165.8],
    'BoilingPoint': [100, 78.37, 80.1, 64.7, 56.08, 110.6, 121.2],
    'Solubility': ['miscible', 'miscible', 'slightly miscible', 'miscible', 'miscible', 'not miscible', 'slightly miscible']
}

#create a DataFrame from the dictionary
df = pd.DataFrame(data)
print(df)
df=df.set_index('Compound') # make a specifric column the index
df

As you can see are notebooks very well readible in jupyter. First we used print(df) and it still looks good. As it had initially indexes as row names we exchanged this with a specific column. 

For the looks:<br>
There are a lot of conversion options, including excel and markdown 


## Task

print a markdown table from this dataframe, hint check df.to_ < tab > and print the resulting string. Looks familiar?

# Selecting, slicing and plotting

Like in arrays we can slice using indexes and booleans. But DataFrames have several other great selection possibilities that is related to their similarity to dictionaries. Which is name/value based slicing.

This selection can be done as part of the value in the cells using e.g. "groupby" or by the indexes, on which we will exclusively focus. (see advanced part for grouping, that is very useful for separate treatments)

## Name - values based slicing

For a DataFrame like above we can use the **name** of a columns to select a column

In [ ]:
df['BoilingPoint']

or multiple names as a list or vector

In [ ]:
df[['BoilingPoint','MolecularWeight']]

The better and more structure way to **"locate"** a certain value/region in the DataFrame is to use the "name base locator" df.loc["name of row","name of column"]<br> 
In a series (a single column) we do of course only need the row index.<br>
If the index or columns we are selecting from are numbers and sorted this allows some really convenient tricks that show some of the usefullness of this.

The following line reads a stored file and sets the first column as index.<br> 
we then use the build in plotting function to look at the read file

In [ ]:
df=pd.read_csv('Data/sinc.dat',index_col=0)
df.plot()

to inspect the contents of the file one can:
* print it by typing **df**
* print the first x (default=5) elements with **df.head()**
* print the last x (default=5) elements with **df.tail()**
* look at the descriptive summery with **df.describe()**

But here we want to slice. So we use df.loc[] and as you would with indexes slices. But the values i give are the values from the axis. What is really neat is that these values do not really need to exist. Pandas will find the nearest value and use that instead.

In [ ]:
df.loc[-1:1,:].plot()

For this to properly work the entries in the index need to be numbers. So we can check the "type" of the  entries and if needed convert them to number with the command "astype".

In [ ]:
df.index # look at the last line. dtype=float64

In [ ]:
df.columns

notice the difference? The values are actually strings and the type is 'object' which means something mixed or non regular.<br>

So we replace the columns with the same values, but now converted into numbers and while we are on it sort them after size. Now we can also slice the second column. (note that the values 6 and 12 are not in the list but the closest has been selected)

In [ ]:
df.columns=df.columns.values.astype(float)
df.sort_index(inplace=True,axis='columns')
df.loc[-1:1,6:12].plot()

This would of course have been very useful for doing calculations on sections of data.

As the example above shows can commands be "stacked". Here we first sliced and then plotted. Important to note is that we have not assigned this temporarily sliced notebook to anything. so we have not written

    df=df.loc[-1:1,6:12]
    df.plot()

The difference is important, as we have not changed df, only create a local copy. We will later use this trick to stack a number of commands. But here this creates a convenient way to make plots. To take full advantage we should assign a name to the columns and the index as a whole.<br> compare the plots to see the difference.

In [ ]:
df.index.name='function x-value in cm'
df.columns.name=r'parameter for the $\frac{sin(x)}{n * x}$ calculation'

fig,ax=plt.subplots(figsize=(8,4))
df.loc[-1:1,6:12].plot(ax=ax,legend=True)

In [ ]:
df

## Task

While this was kind of useful for categorical data, but becomes really useful for e.g. spectroscopic data<br>
Here we read in the measured absorption of silicon, please plot this spectrum only in the visible spectral range

In [ ]:
df=pd.read_csv("Data/Silicon_Absorption.csv",index_col=0)

###### Indexes and values
Both indexes and values in the main matrix are essentially numpy vectors/arrays and can be extracted. 

    data_array=df.values
    index_array=df.index.values
    columns_array=df.columns.values

or set with a list&vector of the same shape:

    df.columns=['Hey I'm column 1','Hey I'm column 2','Hey I'm column 3' ]
    
Indexing (setting or changing of indexes) is one of the big challenges in working with Pandas and will take some time to get used to. However they can have some really cool features as the following example from the **biochemistry** world will show.

First we create two lists with 600000 measured sequences of each 3 or 128 basepairs. 

In [ ]:
testing_list1=np.random.choice(['A','B','C','D'],size=(3,600000))
testing_list1=np.apply_along_axis(''.join, 0, testing_list1)

testing_list2=np.random.choice(['A','B','C','D'],size=(3,600000))
testing_list2=np.apply_along_axis(''.join, 0, testing_list2)

print(testing_list1.shape)
testing_list1[:10]

* then we count how often we have each item in the list using Counter that creates a dictionary with the name and how often the item is in it.
* we then create a pandas series for each
* and calculate the difference between the two

In [ ]:
from collections import Counter
import time

starting_time=time.time()

#count how often is each basepair in sequence
intens1=Counter(testing_list1)
intens2=Counter(testing_list2)

#create series from counts
series1=pd.Series(intens1,index=intens1.keys())
series2=pd.Series(intens2,index=intens2.keys())

intermediary_time=time.time()

#calculate how different are they (in the sum)
diff=series1.subtract(series2, fill_value=0, axis=0)
difference=diff[diff.abs()>0].abs().sum()

final_time=time.time()
print('%g basepairs were different calculated in %.3g ms for %g samples of %g bases'%(difference,(final_time-starting_time)*1000,600000,3))
print('from this time only %.3g ms were spend to joining the base-pairs using a pandas series'%((final_time-intermediary_time)*1000))

Here we use the feature of DataFrames/Series to merge two completly different matrixes. If e.g. one of the keys (here base pairs) does not exist in one of the series it is intelligently added. <br>
<br> The uses are endless, from procedures in chemistry over samples in testing or to the here intelligent comparision.

## Index based and boolean slicing

Of course we have the same methods as numpy. 
with 

    df.iloc[2:4,1:5]
    
we perform an index based slicing. The difference, we always slice the indexes as well as the data.<br>
This means if in the example of the SINC function from above you slice the x-axis (e.g. spectral axis) and the parameter axis. One would do that in numpy or Matlab with:

    array=array[2:4,1:5]
    x_axis=x_axis[2:4]
    y_axis=y_axis[1:5]
    
The same is true for bolean (value) type slicing

    df[df>5] = 0
    
Will for example set all values in the matrix where the intensity is bigger than 5 to 0. We will practice this one we have learned to read Data.

# Reading data with Pandas

reading arbitrary data from a file or the web can be very tedious and difficult. Here we want to use the power of Pandas to simplify this process

Essentially anything that you do _manually_ when extracted data from the internet which is problematic due to several reasons:

1. it's time consuming
2. it's error prone
3. what if the source is updated / corrected?

We have already used Pandas to read files above

    df=pd.read_csv("Data/Silicon_Absorption.csv",index_col=0)
    
Which nearly magically recognizes the shape of the file and reads it. We will train more later. Pandas is a powerful tool for reading and handling data. It can easily handle large data bases; millions to billions of entries. It even can read even larger files bunch by bunch. 

Additionally we can simply point it to the Wikipedia page and it will automatically - and almost magically - detect tables and extract the values.

In [ ]:
tables = pd.read_html("https://en.wikipedia.org/wiki/Hydrophobicity_scales")
p = tables[0] # list of table found. Only one is found on the page. 
p.head() # show the first five rows (the head)

In [ ]:
p.set_index('Amino acid',inplace=True) # we want to use aminoacids as index
p=p['Interface scale, ΔGwif (kcal/mol)']  # and only want one of the columns
p

## Challenges when reading data

Notice the **dtype:object** again? 
turns out Wikipedia uses a weired type of minus signs.
if you try:

    p.astype(float)

it fails. Wikipedia as well as a few machines made in India (Chinese ones usually work) use a different type of minus sign. After replacing the strange minus with a proper minus we can again build the pandas series and now create very nice plots. <br> One thing of note. In my 15 years of using python I have only a handful of times needed to do something like in this example. Each of which used Wikipedia, which was why we wanted to show you the trick.

Starting again by reading and shaping, then in Line 6 we replace the minus sign, after which we can plot (or do some calculations)

In [ ]:
tables = pd.read_html("https://en.wikipedia.org/wiki/Hydrophobicity_scales")
p = tables[0] # list of table found. Only one is found on the page. 
p.set_index('Amino acid',inplace=True) # we want to use aminoacids as index
p=p['Interface scale, ΔGwif (kcal/mol)']  # and only want one of the columns

p=pd.Series([float(s.replace('−', '-')) for s in p.values],index=p.index,name=p.name)
fig,ax=plt.subplots()
ax=p.plot(ax=ax,kind='bar')
ax.set_ylabel(p.name)

**This summarizes the 4 typical challenges when handling data.**

1. Finding the files/plots/sources
2. Telling the program where they are
3. Telling the program how to read the data
4. Cleaning up the data you have read.


## Task

Read the data in " http://www.jensuhlig.de/Kemm30/sinc.dat " or "Data/sinc.dat" using the function df=pd.read_csv() 

For this you can either open the file directly from the internet or download it into your working directory first. 

In general when working with text files: Important **DO NOT** double click but open the file in a better plain text reader. I recommend something like Notepad++ . Look at the file and note (maybe on a paper) 

1. if there is any text before the data that you do not want to read
2. if there are any headers (text before the data that tells what the columns are)
3. what are the separators between the numbers
4. which column (if any) would be good to use as index (think x-axis in plot)

These 4 points translate into the 4 most used options in pd.read_csv() 

    1. skip_rows= 0 or x
    2. header=None or True
    3. sep=',' or '\t' (tab) or '\s+' one or multiple white
    4. index_col = 0 or x or None

If everything worked it should look like this if

<div>
<img src="Data/sinc_reading.jpg" width="600">
</div>

## Task

Produce the same plot for "http://www.jensuhlig.de/Kemm30/sinc_2.dat" or "Data/sinc_2.dat"

### Useful:Dealing with directories

Before we start to load and handle many datafiles we should quickly look into how to handle directories. 
Loading a file from a sub directory needs the filesystem separator, meaning the sign that separates folders and files.

If you are working on "binder" or "garm" then the program of your Jupyter notebook is running on the server. This means that you need to upload your data to this server to be able to use it.

Lets assume that you have your datafiles in a (relative) subfolder with the name "Data" Then during file import you need to replace you filename "sinc.dat" with the path to the file<br>
under windows this means using the backslash "data\sinc.dat"<br>
as however the "\" is a special character we need to use "\\" to create it

In [ ]:
filename='data\\sinc.dat' #under windows
filename='data/sinc.dat' #under MAC and Linux

The library "os" provides some simple tools to make this more comfortable <br>
"os.sep" provides the platform independent separator (on linux and windows alike) <br>
"os.getcwd()" read "get current working directory" can tell you where you currently are<br>
For more complicated path there is the excellent library "pathlib". 

In [ ]:
import os #import the library
filename='data' + os.sep + 'sinc.dat'
filename=os.sep.join(['data','sinc.dat']) #easy to read and independent of which platform

you can also use the features of "jupyter lab" to copy the path<br> 
or finally use a graphical interface to select your file. See this example:

In [ ]:
def get_path():
    from tkinter import filedialog
    import tkinter
    root_window = tkinter.Tk()			
    root_window.withdraw()
    root_window.attributes('-topmost',True)
    root_window.after(1000, lambda: root_window.focus_force())
    complete_path = filedialog.askopenfilename(initialdir=os.getcwd())
    return complete_path
if 0: #so that we can run "run all above"
    get_path()

## Task

The data file " http://www.jensuhlig.de/Kemm30/APS_Copper_SolarCell.dat " "Data/APS_Copper_SolarCell.dat" was measured at a large scale research facility: the "Advanced light source" in Chicago and represents X-ray absorption data. Ignore the first 30 rows (and header) and read all columns from the file. We want to plot the Column "PR" vs the Column "Energy"

If everything worked it should look like this:
<div>
<img src="http://www.jensuhlig.de/Kemm30/APS_reading.jpg" width="300">
</div> 

Now you have multiple choices how to make this work.

1. count the number of rows and slice. The you can either plot by hand or create a pd.Series for the plot.
2. copy Row 30 from the text file and paste it in your Notebook as a long string. Then use the string method "split" to separate the string into a list of column names and replace the automatic column names with these names during the import
3. import the columns and replace after the import the column names. use the set_index we have used prior
4. learn more of the many options, there are always some combinations that lead to success.



Congratulation, this is most likely one of the most difficult files you ever need to read, combining all the techniques you have learned up to now.

# Creation of DataFrames

Dictionaries are often the easiest way to create and name DataFrames other methods are attaching columns to dataframes.

In [ ]:
timen=np.arange(0,60,0.1)   # create a time vector
dicten={}                   # create an empty dictionary to contain the data
for rate in np.arange(1.5,10,1):     # loop over the parameter you want to use and put the current parameter into "rate"
    y=np.exp(-timen/rate)            # create the vector with the y-values 
    dicten['%.1f'%rate]=y            # store the value in the dictionary with the parameter as the key 
df=pd.DataFrame(dicten)              # Now create the dataframe. 
df.index.name='time [s]'             # give the x-axis a name
df.columns.name='rate [mol/s]'       # give the parameter a name
df.name='Concentation'

## Task
1. Create a DataFrame with the index in from of floats from -5 cm to 5 cm in 0.05 steps.
2. in the columns add different gaussians bell curves <br>
    ${\frac {1}{\sqrt {2\pi \sigma ^{2}}}}\operatorname {exp} \left(-{\frac {\left(x-\mu \right)^{2}}{2\sigma ^{2}}}\right)$<br>
with the same central position (mu=0) but different width sigma (0.5,1,2,3,4)
3. use the different sigma as column names
4. using the simplified plotting from above to show them in the same plot

# Statistics with DataFrames

you have already learnt **max**, **min**, **std** **sum**, **mean**, **median**, **describe**. All of these functions can be applied on all or per row, per column,...

A very useful tool can be the groupby functions. Lets look on our original data again

In [ ]:
data = {
    'Compound': ['Water', 'Ethanol', 'Benzene', 'Methanol', 'Acetone', 'Toluene', 'Tetrachloroethylene'],
    'MolecularWeight': [18.015, 46.07, 78.1134, 32.042, 58.08, 92.14, 165.8],
    'BoilingPoint': [100, 78.37, 80.1, 64.7, 56.08, 110.6, 121.2],
    'Solubility': ['miscible', 'miscible', 'slightly miscible', 'miscible', 'miscible', 'not miscible', 'slightly miscible']
}

df = pd.DataFrame(data)
df=df.set_index('Compound') # make a specifric column the index
df

In [ ]:
#we can create groups by any categories. e.g. here the entries in Solubility
groups=df.groupby('Solubility')
#Then we can perform calculations within the groups.
groups['BoilingPoint'].mean()

## Task
Calculate the average molecular weight for the groups

# Bonus: RDKit

**Obs!** you will need to install RDKit ([här](https://www.rdkit.org/docs/Install.html))<br>
you can do that from within this notebook with 
    
    !pip install 
    
and then restart your kernel

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors
m = Chem.MolFromSmiles('c1ccccc1C(=O)O')
m

In [ ]:
print("number of atoms:", m.GetNumAtoms())
for atom in m.GetAtoms():
    print(atom.GetSymbol())
print("number of atoms with hydrogen:", Chem.AddHs(m).GetNumAtoms())
print("Molar mass:",Descriptors.MolWt(m),"g/mol")

In [ ]:
from rdkit.Chem import PandasTools

mollist = []
mollist.append(Chem.MolFromSmiles('O'))
mollist.append(Chem.MolFromSmiles('CO'))
mollist.append(Chem.MolFromSmiles('CCO'))
mollist.append(Chem.MolFromSmiles('CC(=O)C'))
mollist.append(Chem.MolFromSmiles('c1ccccc1'))
mollist.append(Chem.MolFromSmiles('c1ccccc1C'))
mollist.append(Chem.MolFromSmiles('ClC(Cl)C(Cl)Cl'))
df['Structure'] = mollist

PandasTools.RenderImagesInAllDataFrames(images=True)
df

In [ ]:
#solution to the automatic reading
df3b=pd.read_csv("http://www.jensuhlig.de/Kemm30/APS_Copper_SolarCell.dat",skiprows=29,escapechar='L',sep='\s+',index_col=0,usecols=['Energy','PR'])